In [17]:
import pandas as pd
import re
import ast
from ast import literal_eval
import statistics

# Silence the warning
pd.options.mode.chained_assignment = None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Change path to your drive

In [ ]:
cd '/content/drive/MyDrive/Team folder/7. Impact tool/2. Change request PDF/Scoring_algo'

/content/drive/.shortcut-targets-by-id/1Pw7YwAnzS7_XBn0YvLgMGXKg0AN42dNG/Team folder/7. Impact tool/2. Change request PDF/Scoring_algo


### Load data

In [ ]:
# Higg Index
higg = pd.read_csv("higg_msi_scored_manual.csv")
higg.drop('Unnamed: 0', axis=1, inplace = True)
material_types = []
for m in higg['Material']:
    material_types.append(m.lower())
# Good On You
good = pd.read_csv('goodonyou_dataset.csv')
# Farfetch
item_df = pd.read_csv('item.csv')
material_df = pd.read_csv('material.csv')

In [19]:
good

,Unnamed: 0,BrandName,Domain,Rating,OverallRating,Description
0,0,1017-alyx-9sm,Planet,2 out of 5,Not good enough,1017 Alyx 9SM's environment rating is 'not goo...
1,1,1017-alyx-9sm,People,3 out of 5,Not good enough,1017 Alyx 9SM's environment rating is 'not goo...
2,2,1017-alyx-9sm,Animals,2 out of 5,Not good enough,1017 Alyx 9SM's environment rating is 'not goo...
3,0,12-storeez,Planet,2 out of 5,Not good enough,12 STOREEZ's environment rating is 'not good e...
4,1,12-storeez,People,2 out of 5,Not good enough,12 STOREEZ's environment rating is 'not good e...
...,...,...,...,...,...,...
5917,1,zilver,People,3 out of 5,Good,Zilver's environment rating is 'good'. It uses...
5918,2,zilver,Animals,4 out of 5,Good,Zilver's environment rating is 'good'. It uses...
5919,0,zou-xou,Planet,3 out of 5,It's a start,ZOU XOU's environment rating is 'it's a start'...
5920,1,zou-xou,People,3 out of 5,It's a start,ZOU XOU's environment rating is 'it's a start'...


# Define function

In [ ]:
def get_scores():
    
    ################################################ INPUT ################################################

    brand_name = input("Brand name: ")

    # For Farfetch
    item_name = input("Item name/type: ")
    item_loc = item_df[item_df['Product'].str.contains(item_name, case=False)]
    if len(item_loc) > 1:
        print("There are " + str(len(item_loc)) + " items with similar names:")
        print(item_loc['Product'])
        item_index = input("Please select the index of the item: ")
        item_loc = item_df.iloc[int(item_index),:]
        item_weight = float(item_loc['Weight_kg'])

    elif len(item_loc) == 0:
        pass
    else:
        item_weight = float(item_loc['Weight_kg'])

    # Material info
    materials = []
    weights = []
    n = int(input("Total number of materials: ")) 
    for i in range(0, n): 
        materials.append(input("Material name: "))
        weights.append(input("Percentage weight: "))

    ############################################ GETTING SCORES ###############################################

    #making sure the weights are tractable numbers
    for i in range(len(weights)):
        if (weights[i] + ' ').find('%') != -1:
            weights[i] = float(weights[i].strip('% '))/100
        else:
            weights[i] = float(weights[i])/100
    
    #scores
    GlobalWarmingScore = []
    EutrophicationScore = []
    WaterScarcityScore = []
    AbioticResourceDepletionScore = []
    ChemistryScore = []
    OverallScore = []

    #weights of CO2 & water
    CO2Weight = 0.0
    WaterWeight = 0.0

    #looping over every material

    for i in range(len(materials)):
        #check if material has multiple parts
        if len(materials[i].split()) ==  1:
            row = higg[higg['Material'].str.contains(str(materials[i]), case =False)]
        else:
            mat_indiv = materials[i].split()
            if len(mat_indiv) == 2:
                m0 = mat_indiv[0]
                m1 = mat_indiv[1]
                a0 = '.*'
                a1 = '|'
                m_final = m0+a0+m1+a1+m1+a0+m0

            elif len(mat_indiv) == 3:
                m0 = mat_indiv[0]
                m1 = mat_indiv[1]
                m2 = mat_indiv[2]
                a0 = '.*'
                a1 = '|'
                m_final = m0+a0+m1+a0+m2+a1+m2+a0+m1+a0+m0

            row = higg[higg['Material'].str.contains(m_final, case =False)]

        #check length of relevant results
        if len(row) > 1:
            print("There are " + str(len(row)) + " materials associated with " + materials[i])
            print(row['Material'])
            mat_refined = input("Please choose the index of the material: ")
            
            if mat_refined.isdigit():
                row = higg.iloc[int(mat_refined),:]
                material_name = row['Material']
                GlobalWarmingScore.append(weights[i]*float(row['Score_GlobWarm']))
                EutrophicationScore.append(weights[i]*float(row['Score_Eutrophication']))
                WaterScarcityScore.append(weights[i]*float(row['Score_Water']))
                AbioticResourceDepletionScore.append(weights[i]*float(row['Score_ResourceDepletion']))
                ChemistryScore.append(weights[i]*float(row['Score_Chem']))
                OverallScore.append(weights[i]*float(row['Score_Overall']))
                
            else:
                row = higg[higg['Material'].str.contains(mat_refined, case =False)]
    
        elif len(row) == 0:
            print(materials[i] + " can't be found, please refine your search")
            break
            
        else:
            material_name = row['Material']
            GlobalWarmingScore.append(weights[i]*float(row['Score_GlobWarm']))
            EutrophicationScore.append(weights[i]*float(row['Score_Eutrophication']))
            WaterScarcityScore.append(weights[i]*float(row['Score_Water']))
            AbioticResourceDepletionScore.append(weights[i]*float(row['Score_ResourceDepletion']))
            ChemistryScore.append(weights[i]*float(row['Score_Chem']))
            OverallScore.append(weights[i]*float(row['Score_Overall']))
            
        # Farfetch tool
        row_farfetch = material_df[material_df['Material'].str.contains(str(materials[i]), case =False)]

        #check length of relevant results
        if len(row_farfetch) == 0:
            pass
        elif len(item_loc) == 0:
            pass
        else:     
            #get weights of CO2 & water     
            item_kg = float(item_loc['Weight_kg'])
            co2_kg = float(row_farfetch['CO2_kg']) * item_kg * weights[i]
            water_kg = float(row_farfetch['Water_kg']) * item_kg * weights[i]

            CO2Weight += co2_kg
            WaterWeight += water_kg
################################################ OUTPUT ################################################

    #create a DataFrame with our main findings    
    WaterScore = 0.0
    WS, ES = 0.0, 0.0
    for ws in WaterScarcityScore:
        WS += ws
    for es in EutrophicationScore:
        ES += es
    WaterScore = (WS + ES) / 2

    CO2Score = 0.0
    for gs in GlobalWarmingScore:
        CO2Score += gs
    
    # item_name =  str(brand_name) + " " + str(material_name)

    # GoodOnYou score
    good_score_loc = good[good['BrandName'].str.contains(brand_name, case=False)]
    if good_score_loc.empty:
        good_score_loc = good[good['Description'].str.contains(brand_name, case=False)]
        if good_score_loc.empty:
            print('Brand not in Good On You')
            planet_score = None
        else:
            good_score_loc = good_score_loc.drop(columns='Index')
            sc = good_score_loc['Rating'].loc[good_score_loc['Domain'] == 'Planet']
            planet_score = sc.iloc[0][0]
    else:
        good_score_loc = good_score_loc.drop(columns='Index')
        if len(good_score_loc) > 3:
            print("There are " + str(int(len(good_score_loc)/3)) + " brands with similar names:")
            print(good_score_loc['BrandName'].drop_duplicates())
            brand_index = input("Please select the index of the brand: ")
            good_score_loc = good.iloc[int(brand_index),:]
            sc = good_score_loc['Rating']
            planet_score = sc[0][0]
        else:
            sc = good_score_loc['Rating'].loc[good_score_loc['Domain'] == 'Planet']
            planet_score = sc.iloc[0][0]

    scores_table = pd.DataFrame([brand_name, item_name, WaterScore, CO2Score, planet_score, WaterWeight, CO2Weight]).transpose()
    scores_table.columns = ['Brand Name', 'Item Name', 'Water Score', 'CO2 Score', 'GoodOnYou', 'Water Weight', 'CO2 Weight']
    water_score = scores_table['Water Score'].astype(float).round(1)
    co_score = scores_table['CO2 Score'].astype(float).round(1)
    scores_table['Water Score'] = water_score
    scores_table['CO2 Score'] = co_score
    if planet_score is not None:
        scores_table['Overall Score'] = (water_score + co_score + float(planet_score)) / 3
    else:
        scores_table['Overall Score'] = (water_score + co_score) / 2
    overall_score = scores_table['Overall Score'].astype(float).round(1)
    scores_table['Overall Score'] = overall_score
    scores_table = scores_table[['Brand Name', 'Item Name', 'Water Score', 'CO2 Score', 'GoodOnYou', 'Overall Score', 'Water Weight', 'CO2 Weight']]

    print('\n')
    
    # Water litres used
    water_perday = 3.2
    water_permonth = 97.33
    water_pershower = 60.0
    if WaterWeight < 90:
        water_consumption = WaterWeight / water_perday
        print(round(water_consumption), 'days of drinking water')
    elif 89 < WaterWeight < 900:
        water_consumption = WaterWeight / water_permonth
        print(round(water_consumption), 'months of drinking water') 
    elif WaterWeight > 899:
        water_consumption = WaterWeight / water_pershower
        print(round(water_consumption), 'days of showering')
        
    # CO2 kg used
    co2_phone = 128.0
    co2_mile = 2.5
    co2_work = 15
    days_peryear = 365.0
    months_peryear = 12.0
    if CO2Weight < 3:
        co2_consumption = CO2Weight * co2_phone / (days_peryear/months_peryear)
        print(round(co2_consumption), 'months charging your phone')
    elif 2 < CO2Weight < 10:
        co2_consumption = CO2Weight * co2_phone / days_peryear
        print(round(co2_consumption), 'years of phone charged') 
    elif 9 < CO2Weight < 20:
        co2_consumption = CO2Weight * co2_mile
        print(round(co2_consumption), 'miles driven by car') 
    elif 19 < CO2Weight:
        co2_consumption = CO2Weight * co2_mile / co2_work
        print(round(co2_consumption), 'number of car commutes-to-work') 


    return scores_table

## Get Score

In [ ]:
get_scores()

Brand name: riot
Item name/type: cover up
Total number of materials: 1
Material name: nylon
Percentage weight: 100
There are 4 materials associated with nylon
16                                 Nylon fabric
17                       Nylon fabric, recycled
32                 Nylon/Polyamide (PA) plastic
33    Nylon, mechanically recycled, for plastic
Name: Material, dtype: object
Please choose the index of the material: 16
Brand not in Good On You


0 days of drinking water
0 months charging your phone


,Brand Name,Item Name,Water Score,CO2 Score,GoodOnYou,Overall Score,Water Weight,CO2 Weight
0,riot,cover up,3.6,1.2,None,2.4,0,0
